#Instagram Public Post Scraper
Instagram-scraper is an application written in Python3 that scrapes and downloads instagram posts for puv Use responsibly.



##Necessary Libraries
This program was written in the language Python3 on Google Colab. Make sure the following are available for the program to run properly:
- Selenium (3.141.0+)
- chromium-chromedriver

In [ ]:
!pip3 install selenium
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
import time 
import selenium 

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
print("selenium Version = ", selenium.__version__)

     |████████████████████████████████| 911kB 6.3MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/

In [ ]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

hashtag='food'
# browser.get('https://www.instagram.com/explore/tags/'+hashtag)
Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
class InstagramBot():
    def __init__(self):
      self.chrome_options = webdriver.ChromeOptions()
      self.browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
      self.csvScrapedData = [["description", "weight", "location", "user", "time", "image"]]
      self.hashtag = ""
      
    def signIn(self, email, password):
      print("Signing In")
      self.email = email
      self.password = password
      self.browser.get('https://www.instagram.com/accounts/login/')
      emailInput = self.browser.find_elements_by_css_selector('form input')[0]
      passwordInput = self.browser.find_elements_by_css_selector('form input')[1]
      emailInput.send_keys(self.email)
      passwordInput.send_keys(self.password)
      passwordInput.send_keys(Keys.ENTER)
      time.sleep(2)
    
    def scrape(self, url):
      self.browser.get(url)
      self.browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      elements = self.browser.find_elements_by_xpath("//div[@class='v1Nh3 kIKUG  _bz0w']")
      hrefElements = self.browser.find_elements_by_xpath("//div[@class='v1Nh3 kIKUG  _bz0w']/a")
      elements_link = [x.get_attribute("href") for x in hrefElements]
      for elements in elements_link:
        self.scrapePost(elements)
 
    def scrapePost(self, url):
      self.browser.get(url)
      print("Scraping Post: " + url)
      try: 
        location_element = self.browser.find_element_by_xpath("//a[@class='O4GlU']").text
        location_element = location_element.replace(",", " ")
        user_element = self.browser.find_element_by_xpath("//a[@class='FPmhX notranslate nJAzx']")
        user_element_text = user_element.text
        user_element_text = user_element_text.replace(",", " ")
        user_element_link = user_element.get_attribute("href")
        try:
          desc_element = self.browser.find_element_by_xpath("//div[@class='C4VMK']/span")
          desc_text = desc_element.text
          desc_text = desc_text.replace("\n", " ")
          desc_text = desc_text.replace(",", " ")
        except:
          desc_text = " "
          pass
        try: 
          timestamp_element = self.browser.find_element_by_xpath("//div[@class='k_Q0X NnvRN']/a/time")
          timestamp = timestamp_element.get_attribute("datetime")
          timestamp = timestamp.replace("\n", " ")
          timestamp = timestamp.replace(",", " ")
        except:
          timestamp = " "
          pass    
        try:
          likes_element = self.browser.find_element_by_xpath("//a[@class='zV_Nj']/span").text
          likes_element = likes_element.replace(",", "")
          no_of_likes = int(likes_element)
          followerCount = self.findFollowerCount(user_element_link)
          weight = no_of_likes/followerCount
        except:
          weight = 0
          pass
        image_url = self.findImage()
        self.scrapedData = [desc_text, weight ,location_element, user_element_text, timestamp, image_url]
        print(self.scrapedData)
        self.csvScrapedData.append(self.scrapedData)
      except:
        pass
      
       
    def findImage(self):
      image_element = self.browser.find_element_by_xpath("//div[@class='KL4Bh']/img")
      image_element_link = image_element.get_attribute("src")
      return image_element_link

   
    def findFollowerCount(self, userURL):
      self.browser.get(userURL)
      followers_count_int = 0 
      try:
        count_element = self.browser.find_elements_by_xpath("//span[@class='g47SY ']")
        followers_count = count_element[1].get_attribute("title")
        followers_count = followers_count.replace(",", "")
        followers_count_int = int(followers_count)
      except:
          pass    
      return followers_count_int
    
    
    def scrapeWithHashtags(self, hashtags):
      for hashtag in hashtags:
        self.hashtag = hashtag
        print("-----------Scraping the hashtag '" + hashtag +"' -----------")
        url = 'https://www.instagram.com/explore/tags/' + hashtag
        self.scrape(url)
        
    def exportCSVFile(self):
      csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
      with open('csvScrapedData.csv', 'w') as f:
        writer = csv.writer(f, dialect='myDialect')
        for row in self.csvScrapedData:
          writer.writerow(row)
      f.close()
      


In [ ]:
scrape_type = input("Do you want to scrape any private posts [y/n]:\n")
hashtags = ['dog']
bot = InstagramBot()

if scrape_type == "y":
  username = input("What's your email?\n")
  password = input("What's your password?\n")
  bot.signIn(username, password)
elif scrape_type == "n":
  pass
else:
  sys.exit("No valid Input")

bot.scrapeWithHashtags(hashtags)
bot.exportCSVFile()
